## Notes

In [1]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings("ignore")

### Formation
* Cup patters can last 7 weeks to 65 weeks, **most last for three to six months**
* Correction range from 12% to 15%, sometime up to 33%
* Should look for **at least 30% increase in price in the prior uptrend**
* Bottom of the cup should be U shape; downtrend should not be more than 2.5 times the market averages
* Handle area takes more than one or two weeks to form (shakeout); volume may dry up
    * During a bull market, vol generally don't pick up during a correction in the handle
* Handle should be above stock's 10 week MA price line
    * Handle formed below 10 week MA is prone to fail
    * **A proper handle should be contained within 8% to 12% of its peak** during bull markets, unless stock forms a very large cup.

### Pivot Points
* The day's volume when a "pivot point" or "line of least resistance" happen, the day's vol should increase at least 40% to 50% above normal

### Methodology
1. Find 52-Week Max and check if it's recent
 * Definition of recent: x-amount of week
2. Find max price point after the 52-week max and check how far apart the two max prices are

In [11]:
def downloand_stock_daily(stocks_to_dl, startdate, enddate):
    import fix_yahoo_finance as yf
    import pandas as pd
    from pandas_datareader import data as pdr
    yf.pdr_override()
    
    output_df = pd.DataFrame(columns=['Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
    
    for stock in stocks_to_dl:
        try:
            data_tmp = pdr.get_data_yahoo(stock, start=startdate, end=enddate)
            data_tmp.reset_index(inplace=True)
            data_tmp['Ticker'] = stock
            output_df = output_df.append(data_tmp, sort=False)
        except:
            next
    output_df.reset_index(drop=True, inplace=True)
    return output_df

In [3]:
def annual_max_min(df):
    from functools import reduce
    #output_df = pd.DataFrame(columns=['Ticker', '52wk_max', '52wk_min', '52wk_max_date', '52wk_min_date'])
    
    max_df = df.groupby('Ticker').apply(lambda x: x['High'].max()).reset_index(name='52wk_max')
    max_date_df = df.groupby('Ticker').apply(lambda x: x.loc[x['High'].argmax()]['Date']).reset_index(name='52wk_max_date')
    
    min_df = df.groupby('Ticker').apply(lambda x: x['Low'].min()).reset_index(name='52wk_min')
    min_date_df = df.groupby('Ticker').apply(lambda x: x.loc[x['Low'].argmin()]['Date']).reset_index(name='52wk_min_date')
    
    dfs_to_merge = [max_df, max_date_df, min_df, min_date_df]
    output_df = reduce(lambda left, right: pd.merge(left, right, on ='Ticker'), dfs_to_merge)
    return(output_df)

In [4]:
def annual_max_time_delta(annual_max_df):
    import datetime
    current_date = datetime.datetime.today().strftime('%Y-%m-%d')
    
    annual_max_df['52wk_max_date'] = pd.to_datetime(annual_max_df['52wk_max_date'])
    annual_max_df['current_date'] = current_date
    annual_max_df['current_date'] = pd.to_datetime(annual_max_df['current_date'])
    
    annual_max_df['max_time_delta_wks'] = (annual_max_df['current_date'] - annual_max_df['52wk_max_date']).dt.days/7
    return(annual_max_df)

In [5]:
def find_tailing_max(daily_df, annual_max_min_df):
    output_df = pd.DataFrame(columns=['Ticker', 'tailing_max_date', 'pct_change_to_52wk_max', 'time_delta_to_52wk_max_wk'])
    
    for stock in annual_max_min_df['Ticker']:
        max_52wk_date = annual_max_min_df[annual_max_min_df['Ticker'] == stock]['52wk_max_date'].values[0]
        #if the 52wk max date is today, skip
        if datetime.datetime.utcfromtimestamp(max_52wk_date.tolist()/1e9).strftime('%Y-%m-%d') != datetime.datetime.today().strftime('%Y-%m-%d'):
            max_52wk_price = annual_max_min_df[annual_max_min_df['Ticker'] == stock]['52wk_max'].values[0]
        
            post_52wk_max_df = daily_df[(daily_df['Ticker'] == stock) & (daily_df['Date']>max_52wk_date)].reset_index(drop=True)
            post_52wk_max_min_date =post_52wk_max_df.loc[post_52wk_max_df['Low'].argmin]['Date']
            tailing_max_df = daily_df[(daily_df['Ticker'] == stock) & (daily_df['Date']>post_52wk_max_min_date)]
        
            if len(tailing_max_df) !=0:
                tailing_max_date = tailing_max_df.loc[tailing_max_df['High'].argmax]['Date']
                tailing_max_price = tailing_max_df['High'].max()
                output_df = output_df.append(pd.DataFrame({'Ticker': [stock],
                                                          'tailing_max_date': [tailing_max_date],
                                                          'pct_change_to_52wk_max': [(tailing_max_price-max_52wk_price)/max_52wk_price],
                                                          'time_delta_to_52wk_max_wk': [(tailing_max_date - max_52wk_date).days/7]}))
            else: next
        else: next
    return(output_df)

In [179]:
find_tailing_max(df, annual_max_df)

AAPL 2019-01-25 00:00:00 -0.32269668769993265 16.285714285714285
AMPH 2019-01-29 00:00:00 -0.03437767624020879 0.8571428571428571
EXC 2019-01-24 00:00:00 -0.007173016574978222 7.0
FB 2019-01-18 00:00:00 -0.30276280081334744 25.285714285714285


C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\common.py:408: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return maybe_callable(obj, **kwargs)
C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\common.py:408: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return maybe_callable(obj, **kwargs)


In [161]:
annual_max_df

,Ticker,52wk_max,52wk_max_date,52wk_min,52wk_min_date,current_date,max_time_delta_wks
0,AAPL,233.470001,2018-10-03,142.000000,2019-01-03,2019-02-01,17.285714
1,AMPH,22.980000,2019-01-23,14.400000,2018-05-10,2019-02-01,1.285714
2,EXC,47.400002,2018-12-06,35.570000,2018-02-06,2019-02-01,8.142857
3,FB,218.619995,2018-07-25,123.019997,2018-12-24,2019-02-01,27.285714


In [195]:
max_and_second_max_delta(df, annual_max_df)

AAPL 1 days 00:00:00
AMPH 1 days 00:00:00
EXC 6 days 00:00:00
FB 13 days 00:00:00


C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\common.py:408: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return maybe_callable(obj, **kwargs)


In [128]:
annual_max_df

,Ticker,52wk_max,52wk_max_date,52wk_min,52wk_min_date,current_date,max_time_delta_wks
0,AAPL,233.470001,2018-10-03,142.000000,2019-01-03,2019-02-01,17.285714
1,AMPH,22.980000,2019-01-23,14.400000,2018-05-10,2019-02-01,1.285714
2,EXC,47.400002,2018-12-06,35.570000,2018-02-06,2019-02-01,8.142857
3,FB,218.619995,2018-07-25,123.019997,2018-12-24,2019-02-01,27.285714


In [90]:
stocks_to_dl = ['EXC', 'AMPH', 'FB', 'AAPL']
df = downloand_stock_daily(stocks_to_dl, '2018-01-29', '2019-01-29')
annual_max_df = annual_max_min(df)
annual_max_time_delta(annual_max_df)

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  
C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  if __name__ == '__main__':


,Ticker,52wk_max,52wk_max_date,52wk_min,52wk_min_date,current_date,max_time_delta_wks
0,AAPL,233.470001,2018-10-03,142.000000,2019-01-03,2019-02-01,17.285714
1,AMPH,22.980000,2019-01-23,14.400000,2018-05-10,2019-02-01,1.285714
2,EXC,47.400002,2018-12-06,35.570000,2018-02-06,2019-02-01,8.142857
3,FB,218.619995,2018-07-25,123.019997,2018-12-24,2019-02-01,27.285714


# Testing

In [7]:
tickers = pd.read_csv('../data/america_2019-01-30.csv')
tickers = tickers[tickers['Volume']> 1.0e+06].reset_index(drop=True)

In [80]:
#demo_df[demo_df['Ticker'] == 'NXST']

In [13]:
demo_df = downloand_stock_daily(tickers['Ticker'].sample(500).values, startdate='2018-02-01', enddate='2019-02-01')
#demo_df = downloand_stock_daily(['AMPH'], startdate='2018-01-15', enddate='2019-01-15')
annual_max_df_demo = annual_max_min(demo_df)
annual_max_df_demo = annual_max_time_delta(annual_max_df_demo)
import datetime
output_df = find_tailing_max(demo_df, annual_max_df_demo).sort_values(by='pct_change_to_52wk_max', ascending=False)
output_df[(output_df['pct_change_to_52wk_max'] >= -0.03) & (output_df['time_delta_to_52wk_max_wk']>=6)]

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

,Ticker,tailing_max_date,pct_change_to_52wk_max,time_delta_to_52wk_max_wk
0,AVGO,2019-01-30,-0.000365,46.142857
0,NXST,2018-12-04,-0.001003,20.857143
0,DNB,2019-02-01,-0.001034,20.000000
0,DOV,2019-01-31,-0.001773,18.857143
0,ORIT,2019-01-30,-0.006358,51.000000
0,GIL,2019-02-01,-0.006395,8.428571
0,EXPO,2018-11-16,-0.007329,9.000000
0,BATRA,2019-02-01,-0.011994,18.142857
0,NI,2018-12-13,-0.013518,13.000000
0,IPOA,2019-01-08,-0.018465,22.571429


In [18]:
demo_df[demo_df['Ticker'].isin(output_df[(output_df['pct_change_to_52wk_max'] >= -0.03) & (output_df['time_delta_to_52wk_max_wk']>=6)]['Ticker'].values)].groupby('Ticker')['Open'].mean()

Ticker
AEE       61.476111
ATO       89.957659
AVGO     238.581508
AWR       59.230833
BATRA     24.889917
BRKR      31.670714
CTWS      64.996468
DNB      132.622540
DOV       80.267027
EXPO      47.512302
FNSR      18.561548
GCO       42.705119
GIL       29.909325
HRB       26.128849
IPOA       9.998615
KTOS      12.570714
MMS       65.734683
NI        25.263571
NXST      75.154802
ORIT      15.748016
SPLK     106.367182
STNLU     10.146909
TR        30.871828
XEL       47.005159
Name: Open, dtype: float64

In [59]:
annual_max_min_df = annual_max_df_demo
daily_df = demo_df
for stock in annual_max_min_df['Ticker']:
    max_52wk_date = annual_max_min_df[annual_max_min_df['Ticker'] == stock]['52wk_max_date'].values[0]
    
    #if the 52wk max date is today, skip
    if datetime.datetime.utcfromtimestamp(max_52wk_date.tolist()/1e9).strftime('%Y-%m-%d') != datetime.datetime.today().strftime('%Y-%m-%d'):
        max_52wk_price = annual_max_min_df[annual_max_min_df['Ticker'] == stock]['52wk_max'].values[0]
        
        post_52wk_max_df = daily_df[(daily_df['Ticker'] == stock) & (daily_df['Date']>max_52wk_date)].reset_index(drop=True)
        post_52wk_max_min_date =post_52wk_max_df.loc[post_52wk_max_df['Low'].argmin]['Date']
        tailing_max_df = daily_df[(daily_df['Ticker'] == stock) & (daily_df['Date']>post_52wk_max_min_date)]
        
        if len(tailing_max_df) !=0:
            tailing_max_date = tailing_max_df.loc[tailing_max_df['High'].argmax]['Date']
            tailing_max_price = tailing_max_df['High'].max()
            print(stock, tailing_max_date, tailing_max_price, max_52wk_date, max_52wk_price, (tailing_max_price-max_52wk_price)/max_52wk_price, (tailing_max_date - max_52wk_date).days/7)
        else: next
    else: next

C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\common.py:408: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return maybe_callable(obj, **kwargs)
C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\common.py:408: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return maybe_callable(obj, **kwargs)


ABDC 2019-02-01 00:00:00 7.39 2018-02-01T00:00:00.000000000 7.98 -0.07393483709273192 52.142857142857146
ACST 2019-01-11 00:00:00 1.15 2018-10-01T00:00:00.000000000 1.8 -0.36111111111111116 14.571428571428571
ALLE 2019-02-01 00:00:00 86.860001 2018-12-03T00:00:00.000000000 94.300003 -0.078897155496379 8.571428571428571
AOBC 2018-12-07 00:00:00 15.32 2018-09-19T00:00:00.000000000 15.95 -0.039498432601880816 11.285714285714286
ARCW 2019-01-23 00:00:00 1.49 2018-06-20T00:00:00.000000000 2.6 -0.42692307692307696 31.0
AREX 2019-01-10 00:00:00 1.3 2018-02-01T00:00:00.000000000 3.43 -0.6209912536443148 49.0
AT 2019-02-01 00:00:00 2.55 2019-01-22T00:00:00.000000000 2.6 -0.019230769230769332 1.4285714285714286
AXNX 2018-12-11 00:00:00 16.889999 2018-10-31T00:00:00.000000000 17.0 -0.006470647058823557 5.857142857142857
BMTC 2019-01-18 00:00:00 38.650002 2018-07-27T00:00:00.000000000 50.349998 -0.23237331608235612 25.0
BZH 2019-02-01 00:00:00 12.9 2018-02-01T00:00:00.000000000 19.23 -0.3291731669

In [26]:
datetime.datetime.today().strftime('%Y-%m-%d')

'2019-02-01'

In [24]:
datetime.datetime.today().strftime('%Y-%m-%d')

'2019-02-01'

'2019-02-01'

In [49]:
datetime.datetime.today().strftime('%Y-%m-%d')

datetime.datetime(2019, 2, 1, 21, 57, 50, 66897)

In [13]:
post_52wk_max_df.loc[post_52wk_max_df['Low'].argmin]

C:\Users\Xi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\common.py:408: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return maybe_callable(obj, **kwargs)


ValueError: attempt to get argmin of an empty sequence

* check for uptrending stock but don't have a handle yet, and if the handle broke
* use volume to screen out
* if tailing max date is not recent - disqualified

* troubleshooting ticker:
    * USDP
    * ATRI
    * KRYS
    * OTEX
    * IMOS
    * WIT
    * SJR
    * IPOA
   